In [1]:
#Carrega bibliotecas do Pyspark
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler


In [2]:
#Conecta a tabela criada no Databricks
dataset = spark.table("bank")
cols = dataset.columns

#display(dataset)

In [3]:
categoricalColumns = ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    # encoder = OneHotEncoderEstimator(inputCol=categoricalCol + "Index", outputCol=categoricalCol + "classVec")
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    # Add stages.  These are not run here, but will run all at once later on.
    stages += [stringIndexer, encoder]

In [4]:
# Converte as váriaveis categóricas em numéricas (Indices)
label_stringIdx = StringIndexer(inputCol="y", outputCol="label")
stages += [label_stringIdx]

In [5]:
# Transforma todos os features em vetores usando VectorAssembler
numericCols = ["age", "balance", "day", "duration", "campaign", "pdays", "previous"]
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

dataset

Out[ 8 ]: DataFrame[age: int, job: string, marital: string, education: string, default: string, balance: int, housing: string, loan: string, contact: string, day: int, month: string, duration: int, campaign: int, pdays: int, previous: int, poutcome: string, y: string]

In [6]:
#Carga da função de Regressão Logística
from pyspark.ml.classification import LogisticRegression
  
partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(dataset)
preppedDataDF = pipelineModel.transform(dataset)


In [7]:
# Realiza um fit nos dados de treinamento
lrModel = LogisticRegression().fit(preppedDataDF)

# Curva ROC para os dados de treinamento
display(lrModel, preppedDataDF, "ROC")

False Positive Rate,True Positive Rate,Threshold
0.0,0.0,0.9102080989617555
0.0,0.1111111111111111,0.9102080989617555
0.011904761904761904,0.1111111111111111,0.8727759997784728
0.011904761904761904,0.2222222222222222,0.7336536465107624
0.023809523809523808,0.2222222222222222,0.7259079653018172
0.03571428571428571,0.2222222222222222,0.6134736427382863
0.03571428571428571,0.3333333333333333,0.5949502237042397
0.047619047619047616,0.3333333333333333,0.5214377378376418
0.05952380952380952,0.3333333333333333,0.4940334048594755
0.07142857142857142,0.3333333333333333,0.3992205115280179


In [8]:
display(lrModel, preppedDataDF)

fitted values,residuals
-4.463155554981384,-0.011394601680481042
-2.5914586517811085,-0.06969015408985259
-4.970051147367216,-0.006894922898203429
-4.894938299514752,-0.007428771318361844
-4.464246210287158,-0.011382322249983277
-5.007168775932872,-0.006645360644032065
-4.730144389014285,-0.008747993919328786
-2.2227135270524596,-0.0977292688833315
-4.38058025598015,-0.012363327533481577
-3.3834761128355844,-0.032815887085159756


In [9]:
# Seleciona somente as colunas relevantes
selectedcols = ["label", "features"] + cols
dataset = preppedDataDF.select(selectedcols)
display(dataset)

label,features,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0.0,"List(0, 42, List(1, 11, 14, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 58.0, 2143.0, 5.0, 261.0, 1.0, -1.0))",58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
0.0,"List(0, 42, List(2, 12, 13, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 44.0, 29.0, 5.0, 151.0, 1.0, -1.0))",44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
0.0,"List(0, 42, List(7, 11, 13, 16, 17, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 33.0, 2.0, 5.0, 76.0, 1.0, -1.0))",33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
0.0,"List(0, 42, List(0, 11, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 47.0, 1506.0, 5.0, 92.0, 1.0, -1.0))",47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
0.0,"List(0, 42, List(12, 16, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 33.0, 1.0, 5.0, 198.0, 1.0, -1.0))",33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
0.0,"List(0, 42, List(1, 11, 14, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 35.0, 231.0, 5.0, 139.0, 1.0, -1.0))",35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
0.0,"List(0, 42, List(1, 12, 14, 16, 17, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 28.0, 447.0, 5.0, 217.0, 1.0, -1.0))",28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
0.0,"List(0, 42, List(7, 14, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 42.0, 2.0, 5.0, 380.0, 1.0, -1.0))",42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
0.0,"List(0, 42, List(5, 11, 15, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 58.0, 121.0, 5.0, 50.0, 1.0, -1.0))",58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
0.0,"List(0, 42, List(2, 12, 13, 16, 17, 18, 20, 21, 32, 35, 36, 37, 38, 39, 40), List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 43.0, 593.0, 5.0, 55.0, 1.0, -1.0))",43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


In [10]:
### Divide os dados entre treinamento e teste, com 30% dos dados para treinamento e o restante para testes
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())

31645
13566

In [11]:
#Carrega a função de Regressão Logística para os dados de treinamento
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [12]:
#Salva o modelo
lrModel.write().overwrite().save("modelo_lr1")

In [13]:
# Realiza predições baseadas no modelo
predictions = lrModel.transform(testData)
predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

+---+-----------+-----+--------------------+----------+--------------------+
age| job|label| rawPrediction|prediction| probability|
+---+-----------+-----+--------------------+----------+--------------------+
 27|blue-collar| 0.0|[-0.7084084883232...| 1.0|[0.32995060141536...|
 28|blue-collar| 0.0|[3.03441618670908...| 0.0|[0.95410494032020...|
 28|blue-collar| 0.0|[3.56645690096780...| 0.0|[0.97252066083067...|
 28|blue-collar| 0.0|[3.43452774764034...| 0.0|[0.96876635995374...|
 29|blue-collar| 0.0|[4.58577436079569...| 0.0|[0.98990705470859...|
 29|blue-collar| 0.0|[3.65304102671244...| 0.0|[0.97474227434795...|
 29|blue-collar| 0.0|[4.02639020441654...| 0.0|[0.98247403128249...|
 29|blue-collar| 0.0|[3.76883625132564...| 0.0|[0.97744171482534...|
 30|blue-collar| 0.0|[3.75304935620615...| 0.0|[0.97709098685331...|
 30|blue-collar| 0.0|[3.38390912971966...| 0.0|[0.96719785365978...|
+---+-----------+-----+--------------------+----------+--------------------+
only showing top 10 rows

In [14]:
# Visualiza as predições e probabilidades do modelo
selected = predictions.select("label", "prediction", "rawPrediction", "probability", "age", "job", "education")
display(selected)

label,prediction,rawPrediction,probability,age,job,education
0.0,1.0,"List(1, 2, List(), List(-0.7084084883232, 0.7084084883232))","List(1, 2, List(), List(0.32995060141536453, 0.6700493985846354))",27,blue-collar,secondary
0.0,0.0,"List(1, 2, List(), List(3.034416186709082, -3.034416186709082))","List(1, 2, List(), List(0.9541049403202051, 0.04589505967979488))",28,blue-collar,secondary
0.0,0.0,"List(1, 2, List(), List(3.5664569009678067, -3.5664569009678067))","List(1, 2, List(), List(0.9725206608306798, 0.027479339169320268))",28,blue-collar,secondary
0.0,0.0,"List(1, 2, List(), List(3.434527747640349, -3.434527747640349))","List(1, 2, List(), List(0.9687663599537482, 0.03123364004625187))",28,blue-collar,secondary
0.0,0.0,"List(1, 2, List(), List(4.585774360795694, -4.585774360795694))","List(1, 2, List(), List(0.9899070547085939, 0.01009294529140608))",29,blue-collar,secondary
0.0,0.0,"List(1, 2, List(), List(3.653041026712444, -3.653041026712444))","List(1, 2, List(), List(0.9747422743479525, 0.02525772565204751))",29,blue-collar,secondary
0.0,0.0,"List(1, 2, List(), List(4.026390204416549, -4.026390204416549))","List(1, 2, List(), List(0.9824740312824931, 0.017525968717507005))",29,blue-collar,secondary
0.0,0.0,"List(1, 2, List(), List(3.7688362513256486, -3.7688362513256486))","List(1, 2, List(), List(0.977441714825347, 0.022558285174653088))",29,blue-collar,secondary
0.0,0.0,"List(1, 2, List(), List(3.7530493562061533, -3.7530493562061533))","List(1, 2, List(), List(0.9770909868533164, 0.022909013146683605))",30,blue-collar,secondary
0.0,0.0,"List(1, 2, List(), List(3.3839091297196613, -3.3839091297196613))","List(1, 2, List(), List(0.9671978536597848, 0.032802146340215145))",30,blue-collar,secondary


In [15]:
#Carrega modelo avaliador
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

Out[ 26 ]: 0.9034982418698769

In [16]:
evaluator.getMetricName()

Out[ 27 ]: 'areaUnderROC'

In [17]:
#print(lr.explainParams())

In [18]:
#Carrega o modelo avaliador para Cross-Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [19]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [20]:
#salva o modelo
#cvModel.save("modelo_cv")
cvModel.write().overwrite().save("modelo_cv")

In [21]:
predictions = cvModel.transform(testData)
print(testData)
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

DataFrame[label: double, features: vector, age: int, job: string, marital: string, education: string, default: string, balance: int, housing: string, loan: string, contact: string, day: int, month: string, duration: int, campaign: int, pdays: int, previous: int, poutcome: string, y: string]
 Out[ 31 ]: 0.9033687768969345

In [22]:
print('Model Intercept: ', cvModel.bestModel.intercept)

Model Intercept: -2.051544917694723

In [23]:
weights = cvModel.bestModel.coefficients
weights = [(float(w),) for w in weights]  # convert numpy type to float, and to tuple
weightsDF = sqlContext.createDataFrame(weights, ["Feature Weight"])
display(weightsDF)

Feature Weight
-0.22222472762057235
-0.04162489880504778
-0.0024997744071399487
0.20147442863588244
-0.09340987917053635
0.4235262987344917
-0.1815665968419371
-0.17803569470157413
0.04541524873504915
-0.36081228145405847


In [24]:
from pyspark.mllib.evaluation import MulticlassMetrics

# View best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction").rdd
metrics = MulticlassMetrics(selected)
print(metrics.confusionMatrix())

# Overall statistics
precision = metrics.precision()
recall = metrics.recall()
f1Score = metrics.fMeasure()
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)


DenseMatrix([[ 11705., 1140.],
 [ 263., 458.]])
Summary Stats
Precision = 0.8965796845053811
Recall = 0.8965796845053811
F1 Score = 0.8965796845053811

In [25]:
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType
from pyspark.ml.linalg import Vectors
from pyspark.mllib.linalg import SparseVector

modelo = LogisticRegressionModel.load("modelo_lr1")
#print(trainingData.first())
     
data1 = [(0, Vectors.sparse(42,[1,11,14,16,17,18,20,21,32,35,36,37,38,39,40],[1,1,1,1,1,1,1,1,1,58,2143,5,261,1,-1]), 24, "blue-collar", "married", "secondary", "no", 1470, "yes", "no", "cellular", 12, "may", 212, 1, -1, 0, "unknown")]
data2 = spark.createDataFrame(data1, ["label", "features"])
 
predictions = modelo.transform(data2)

print(predictions)

selected = predictions.select("label", "prediction", "rawPrediction", "probability")
display(selected)

label,prediction,rawPrediction,probability
0,0.0,"List(1, 2, List(), List(4.025552515199859, -4.025552515199859))","List(1, 2, List(), List(0.9824596014405852, 0.017540398559414774))"
